In [1]:
# Import library for load data
import ray
ray.init(num_cpus=8)
import modin.pandas as pd
#import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
import seaborn as sns
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from category_encoders import OrdinalEncoder
from numpy import genfromtxt
import sys
import csv
# Import deeplearning library
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms 
import matplotlib.pyplot as plt
import time
import os
import glob
import copy
import numpy as np 
from torch.utils.tensorboard import SummaryWriter
from CustomFunction import *
from model_define import *


2023-07-22 22:32:11,372	INFO worker.py:1636 -- Started a local Ray instance.
2023-07-22 22:32:14.473557: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
folder = '/home/jun/CICIDS/dataset_all'
csv_file = csv_file_list(folder)
for folder in csv_file.keys():
    print(folder)
    for file in csv_file[folder]:
        print(f'\t{file}')

Test
	/home/jun/CICIDS/dataset_all/CICIDS2019/Test/Syn.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Test/UDP.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Test/Portmap.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Test/UDPLag.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Test/MSSQL.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Test/NetBIOS.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Test/LDAP.csv
Train
	/home/jun/CICIDS/dataset_all/CICIDS2019/Train/DrDoS_UDP.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Train/Syn.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Train/DrDoS_NetBIOS.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Train/UDPLag.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Train/TFTP.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Train/DrDoS_LDAP.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Train/DrDoS_MSSQL.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Train/DrDoS_SSDP.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Train/DrDoS_NTP.csv
	/home/jun/CICIDS/dataset_all/CICIDS2019/Train/DrDoS_S

In [3]:
# Define CUDA device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
CICIDS2019_columns = ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP',
                'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
                'Total Fwd Packets', 'Total Backward Packets',
                'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
                'Fwd Packet Length Max', 'Fwd Packet Length Min',
                'Fwd Packet Length Mean', 'Fwd Packet Length Std',
                'Bwd Packet Length Max', 'Bwd Packet Length Min',
                'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
                'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
                'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
                'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
                'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
                'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
                'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
                'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
                'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
                'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
                'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
                'Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
                'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
                'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
                'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
                'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
                'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                'Idle Std', 'Idle Max', 'Idle Min', 'SimillarHTTP', 'Inbound', 'Label']

CICIDS2018_columns = ['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port',
                'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
                'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
                'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
                'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
                'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
                'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
                'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
                'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
                'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
                'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
                'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
                'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
                'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
                'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
                'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg',
                'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg',
                'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Subflow Fwd Pkts',
                'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts',
                'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Act Data Pkts',
                'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max',
                'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min', 'Label']

CICIDS2018_columns_fixed = ['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
                'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
                'Total Fwd Packets', 'Total Backward Packets',
                'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
                'Fwd Packet Length Max', 'Fwd Packet Length Min',
                'Fwd Packet Length Mean', 'Fwd Packet Length Std',
                'Bwd Packet Length Max', 'Bwd Packet Length Min',
                'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
                'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
                'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
                'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
                'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
                'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
                'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
                'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
                'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
                'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
                'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
                'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
                'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
                'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
                'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
                'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
                'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                'Idle Std', 'Idle Max', 'Idle Min', 'Label']

CICIDS2018_columns_fixed_2 = [
                'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
                'Total Fwd Packets', 'Total Backward Packets',
                'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
                'Fwd Packet Length Max', 'Fwd Packet Length Min',
                'Fwd Packet Length Mean', 'Fwd Packet Length Std',
                'Bwd Packet Length Max', 'Bwd Packet Length Min',
                'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
                'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
                'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
                'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
                'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
                'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
                'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
                'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
                'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
                'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
                'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
                'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
                'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
                'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
                'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
                'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
                'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                'Idle Std', 'Idle Max', 'Idle Min', 'Label']

CICIDS2017_columns = ['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
                'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
                'Total Fwd Packets', 'Total Backward Packets',
                'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
                'Fwd Packet Length Max', 'Fwd Packet Length Min',
                'Fwd Packet Length Mean', 'Fwd Packet Length Std',
                'Bwd Packet Length Max', 'Bwd Packet Length Min',
                'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
                'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
                'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
                'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
                'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
                'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
                'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
                'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
                'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
                'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
                'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
                'Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
                'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
                'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
                'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
                'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
                'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                'Idle Std', 'Idle Max', 'Idle Min', 'Label']

print("CICIDS2019 columns:", len(CICIDS2019_columns))
print("CICIDS2018 columns: {} and {}".format(len(CICIDS2018_columns_fixed), len(CICIDS2018_columns_fixed_2)))
print("CICIDS2017 columns:", len(CICIDS2017_columns))

CICIDS2019 columns: 88
CICIDS2018 columns: 84 and 80
CICIDS2017 columns: 85


In [5]:
print('-------------ANALYZE DATASET-------------')
data = {}
for dataset in csv_file.keys():
    data_dict = {}
    print(f"Start loading folder {dataset}")
    for file in csv_file[dataset]:
        print(f"\tLoading {file}")
        data_dict.update({file.split('/')[-1].split('.')[0]:pd.read_csv(file, encoding='cp1252')})
        print("\tDONE!")
    data.update({dataset:data_dict})
print("ALL DONE!")
    

-------------ANALYZE DATASET-------------
Start loading folder Test
	Loading /home/jun/CICIDS/dataset_all/CICIDS2019/Test/Syn.csv


Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


	DONE!
	Loading /home/jun/CICIDS/dataset_all/CICIDS2019/Test/UDP.csv
	DONE!
	Loading /home/jun/CICIDS/dataset_all/CICIDS2019/Test/Portmap.csv
	DONE!
	Loading /home/jun/CICIDS/dataset_all/CICIDS2019/Test/UDPLag.csv
	DONE!
	Loading /home/jun/CICIDS/dataset_all/CICIDS2019/Test/MSSQL.csv
	DONE!
	Loading /home/jun/CICIDS/dataset_all/CICIDS2019/Test/NetBIOS.csv
	DONE!
	Loading /home/jun/CICIDS/dataset_all/CICIDS2019/Test/LDAP.csv
	DONE!
Start loading folder Train
	Loading /home/jun/CICIDS/dataset_all/CICIDS2019/Train/DrDoS_UDP.csv
	DONE!
	Loading /home/jun/CICIDS/dataset_all/CICIDS2019/Train/Syn.csv
	DONE!
	Loading /home/jun/CICIDS/dataset_all/CICIDS2019/Train/DrDoS_NetBIOS.csv
	DONE!
	Loading /home/jun/CICIDS/dataset_all/CICIDS2019/Train/UDPLag.csv
	DONE!
	Loading /home/jun/CICIDS/dataset_all/CICIDS2019/Train/TFTP.csv
	DONE!
	Loading /home/jun/CICIDS/dataset_all/CICIDS2019/Train/DrDoS_LDAP.csv
	DONE!
	Loading /home/jun/CICIDS/dataset_all/CICIDS2019/Train/DrDoS_MSSQL.csv
	DONE!
	Loading /hom

RayTaskError(UnicodeDecodeError): [36mray::_deploy_ray_func()[39m (pid=968218, ip=202.25.82.68)
  File "/home/jun/miniconda3/envs/py311/lib/python3.11/site-packages/modin/core/execution/ray/common/engine_wrapper.py", line 44, in _deploy_ray_func
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/jun/miniconda3/envs/py311/lib/python3.11/site-packages/modin/logging/logger_decorator.py", line 128, in run_and_log
    return obj(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/jun/miniconda3/envs/py311/lib/python3.11/site-packages/modin/core/storage_formats/pandas/parsers.py", line 351, in parse
    return PandasParser.generic_parse(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jun/miniconda3/envs/py311/lib/python3.11/site-packages/modin/logging/logger_decorator.py", line 128, in run_and_log
    return obj(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/jun/miniconda3/envs/py311/lib/python3.11/site-packages/modin/core/stora

In [ ]:
for dataset, data_dict in data.items():
    print(dataset)
    for csv_name, csv_data in data_dict.items():
        print("\t{}: {}".format(len(csv_data.columns),csv_name))
        print("\t", )

In [ ]:
# Fix columns name of all dataset
for dataset, data_dict in data.items():
    for csv_name, csv_data in data_dict.items():
        csv_data = fix_columns_name(csv_data)
for csv_name, csv_data in data['CICIDS2018'].items():
    try:
        csv_data.columns = CICIDS2018_columns_fixed
    except:
        csv_data.columns = CICIDS2018_columns_fixed_2

KeyError: 'CICIDS2018'

In [ ]:
for dataset, data_dict in data.items():
    print(dataset)
    for csv_name, csv_data in data_dict.items():
        print("\t", csv_name)
        print(set(csv_data['Label']))

In [ ]:
for dataset, data_dict in data.items():
    print(dataset)
    for csv_name, csv_data in data_dict.items():
        print("\t", csv_name)
        this_columns = csv_data.columns
        not_included = []
        for column in CICIDS2019_columns:
            if column not in this_columns:
                not_included.append(column)
        print(f"\t Not included {len(not_included)} items with: {not_included}")

Test
	 Syn
	 Not included 0 items with: []
	 UDP
	 Not included 0 items with: []
	 Portmap
	 Not included 0 items with: []
	 UDPLag
	 Not included 0 items with: []
	 MSSQL
	 Not included 0 items with: []
	 NetBIOS
	 Not included 0 items with: []
	 LDAP
	 Not included 0 items with: []
Train
	 DrDoS_UDP
	 Not included 0 items with: []
	 Syn
	 Not included 0 items with: []
	 DrDoS_NetBIOS
	 Not included 0 items with: []
	 UDPLag
	 Not included 0 items with: []
	 TFTP
	 Not included 0 items with: []
	 DrDoS_LDAP
	 Not included 0 items with: []
	 DrDoS_MSSQL
	 Not included 0 items with: []
	 DrDoS_SSDP
	 Not included 0 items with: []
	 DrDoS_NTP
	 Not included 0 items with: []
	 DrDoS_SNMP
	 Not included 0 items with: []
	 DrDoS_DNS
	 Not included 0 items with: []
CICIDS2017
	 Monday-WorkingHours
	 Not included 3 items with: ['Unnamed: 0', 'SimillarHTTP', 'Inbound']
	 Wednesday-workingHours
	 Not included 3 items with: ['Unnamed: 0', 'SimillarHTTP', 'Inbound']
	 Friday-WorkingHours-Morning


In [ ]:
for dataset, data_dict in data.items():
    print(dataset)
    for csv_name, csv_data in data_dict.items():
        print("\t", csv_name)
        this_columns = csv_data.columns
        not_included = []
        for column in this_columns:
            if column not in CICIDS2018_columns_fixed_2:
                not_included.append(column)
        print(f"\t Drop {len(not_included)} items: {not_included}")

Test
	 Syn
	 Drop 8 items: ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Fwd Header Length.1', 'SimillarHTTP', 'Inbound']
	 UDP
	 Drop 8 items: ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Fwd Header Length.1', 'SimillarHTTP', 'Inbound']
	 Portmap
	 Drop 8 items: ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Fwd Header Length.1', 'SimillarHTTP', 'Inbound']
	 UDPLag
	 Drop 8 items: ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Fwd Header Length.1', 'SimillarHTTP', 'Inbound']
	 MSSQL
	 Drop 8 items: ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Fwd Header Length.1', 'SimillarHTTP', 'Inbound']
	 NetBIOS
	 Drop 8 items: ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Fwd Header Length.1', 'SimillarHTTP', 'Inbound']
	 LDAP
	 Drop 8 items: ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Fwd Header Length.1', 

In [ ]:
CICIDS2019_drop_list = ['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Timestamp', 'Fwd Header Length.1', 'SimillarHTTP', 'Inbound']
CICIDS2018_drop_list = ['Destination Port', 'Timestamp']
CICIDS2018_drop_list_2 = ['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Timestamp']
CICIDS2017_drop_list = ['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Timestamp', 'Fwd Header Length.1']

In [ ]:
for csv_name, csv_data in data['CICIDS2018'].items():
    try:
        csv_data.drop(CICIDS2018_drop_list, inplace=True, axis=1)
    except:
        csv_data.drop(CICIDS2018_drop_list_2, inplace=True, axis=1)
        print(f"Error {csv_name}")


In [ ]:
for csv_name, csv_data in data['CICIDS2017'].items():
    try:
        csv_data.drop(CICIDS2017_drop_list, inplace=True, axis=1)
    except:
        print(f"Error {csv_name}")

In [ ]:
for csv_name, csv_data in data['Train'].items():
    try:
        csv_data.drop(CICIDS2019_drop_list, inplace=True, axis=1)
    except:
        print(f"Error {csv_name}")

In [ ]:
PATH = '/home/jun/CICIDS/fixed_dataset/'
for dataset, data_dict in data.items():
    print('Writing to {}'.format(dataset))
    merged = pd.concat(data_dict.values(), ignore_index=True, sort=False)
    merged_PATH =PATH + dataset + '.csv'
    print(f'Writing to {merged_PATH}')
    merged.to_csv(merged_PATH,index=False) 
    for csv_name, csv_data in data_dict.items():
        file_PATH =PATH + dataset + '/' + csv_name + '.csv' 
        print(f'\tWriting to {file_PATH}')
        csv_data.to_csv(file_PATH,index=False)
        print('\tDONE!!')
    print('ALL DONE!')

Writing to Test
	Writing to /home/jun/CICIDS/fixed_dataset/Test/Syn.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Test/UDP.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Test/Portmap.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Test/UDPLag.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Test/MSSQL.csv


(raylet) Spilled 132192 MiB, 1987 objects, write throughput 1343 MiB/s.


	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Test/NetBIOS.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Test/LDAP.csv
	DONE!!
ALL DONE!
Writing to Train
	Writing to /home/jun/CICIDS/fixed_dataset/Train/DrDoS_UDP.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Train/Syn.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Train/DrDoS_NetBIOS.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Train/UDPLag.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Train/TFTP.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Train/DrDoS_LDAP.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Train/DrDoS_MSSQL.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Train/DrDoS_SSDP.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Train/DrDoS_NTP.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Train/DrDoS_SNMP.csv
	DONE!!
	Writing to /home/jun/CICIDS/fixed_dataset/Train/DrDoS_DNS.csv
	DONE!!
ALL DONE!
Writing to CICIDS2017
	Writing to /home/jun/CI